In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import collections

import tweepy as tw
import nltk
from nltk.corpus import stopwords
import re
import networkx

In [ ]:
#pip install tweepy

In [ ]:
import warnings
warnings.filterwarnings("ignore")

sns.set(font_scale=1.5)
sns.set_style("whitegrid")

In [ ]:
#add your keys
consumer_key= ''
consumer_secret= ''
access_token= ''
access_token_secret= ''

In [ ]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [ ]:
search_term = "#farmers-filter:retweets"

tweets = tw.Cursor(api.search,
                   q=search_term,
                   lang="en",
                   since='2021-01-04').items(1000)

all_tweets = [tweet.text for tweet in tweets]

all_tweets[:2]

In [ ]:
#pip install twython

In [ ]:
import json

# Enter your keys/secrets as strings in the following fields
credentials = {}
credentials['CONSUMER_KEY'] = ''
credentials['CONSUMER_SECRET'] = ''
credentials['ACCESS_TOKEN'] = ''
credentials['ACCESS_SECRET'] = ''

# Save the credentials object to file
with open("twitter_credentials.json", "w") as file:
    json.dump(credentials, file)

In [ ]:
# Import the Twython class
from twython import Twython
import json

# Load credentials from json file
with open("twitter_credentials.json", "r") as file:
    creds = json.load(file)

# Instantiate an object
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])

# Create our query
query = {'q': '#farmers',
        'result_type': 'popular',
        'count': 100,
       'lang': 'en',
        }

import pandas as pd
from IPython.display import display, HTML


# Search tweets
dict_ = {'TwitterUser': [], 'Date': [], 'Tweet Text': [], 'Likes': []}
for status in python_tweets.search(**query)['statuses']:
    dict_['TwitterUser'].append(status['user']['screen_name'])
    dict_['Date'].append(status['created_at'])
    dict_['Tweet Text'].append(status['text'])
    dict_['Likes'].append(status['favorite_count'])
    

In [ ]:
# Structure data in a pandas DataFrame for easier manipulation
print(query)
df = pd.DataFrame(dict_)
df.sort_values(by='Likes', inplace=True, ascending=False)
display(HTML(df.to_html(index=False)))

In [ ]:
##TWITTER STREAMING

In [ ]:
from twython import TwythonStreamer
import csv

# Filter out unwanted data
def process_tweet(tweet):
    d = {}
    d['hashtags'] = [hashtag['text'] for hashtag in tweet['entities']['hashtags']]
    d['text'] = tweet['text']
    d['user'] = tweet['user']['screen_name']
    d['user_loc'] = tweet['user']['location']
    return d
    
    
# Create a class that inherits TwythonStreamer
class MyStreamer(TwythonStreamer):     

    # Received data
    def on_success(self, data):

        # Only collect tweets in English
        #if data['lang'] == 'en':
            tweet_data = process_tweet(data)
            self.save_to_csv(tweet_data)

    # Problem with the API
    def on_error(self, status_code, data):
        print(status_code, data)
        self.disconnect()
        
    # Save each tweet to csv file
    def save_to_csv(self, tweet):
        with open(r'Tweets_saved.csv', 'a') as file:
            writer = csv.writer(file)
            writer.writerow(list(tweet.values()))

In [ ]:
# Instantiate from our streaming class
stream = MyStreamer(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'], 
                    creds['ACCESS_TOKEN'], creds['ACCESS_SECRET'])
# Start the stream
stream.statuses.filter(track='#farmers')

In [ ]:
# NEXT STEPS LOOKING AT CSV FILE
import pandas as pd
tweets = pd.read_csv("Tweets_saved.csv",header=None)
tweets=tweets[:]
tweets=tweets.drop_duplicates() ## drop duplicate rows
len(tweets)

In [ ]:
from collections import Counter
import ast

tweets = pd.read_csv("saved_tweets.csv",header=None)

# Extract hashtags and put them in a list
list_hashtag_strings = [entry for entry in tweets[0]]
list_hashtag_lists = ast.literal_eval(','.join(list_hashtag_strings))
hashtag_list = [ht.lower() for list_ in list_hashtag_lists for ht in list_]

# Count most common hashtags
counter_hashtags = Counter(hashtag_list)
counter_hashtags.most_common(20)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

clean_tweets_nsw = pd.DataFrame(counter_hashtags.most_common(20),
                             columns=['words', 'count'])

fig, ax = plt.subplots(figsize=(8, 8))

# Plot horizontal bar graph
clean_tweets_nsw.sort_values(by='count').plot.barh(x='words',
                      y='count',
                      ax=ax,
                      color="purple")

ax.set_title("Hastags Trending on Twitter")

plt.show()

In [ ]:
from geopy.geocoders import Nominatim
import gmplot
import numpy as np

lists=['india', 'canada', 'earth', 'worldwide', 'us', 'united states', 'hell', 'twitter','uk']

geolocator = Nominatim(user_agent="aj-application")

# Go through all tweets and add locations to 'coordinates' dictionary
coordinates = {'latitude': [], 'longitude': []}
for count, user_loc in enumerate(tweets[3]):
    try:
        #if user_loc.isnull()==FALSE:
        #print(user_loc is np.nan)
        if user_loc is np.nan:
            pass
        if user_loc.lower() in lists:
            pass
        else:
            location = geolocator.geocode(user_loc)
            #print(user_loc)

        # If coordinates are found for location
        if location:
            coordinates['latitude'].append(location.latitude)
            coordinates['longitude'].append(location.longitude)
            
    # If too many connection requests
    except:
        pass


In [ ]:
# Instantiate and center a GoogleMapPlotter object to show our map

gmap = gmplot.GoogleMapPlotter(30, 0, 3,apikey='apikey')

# Insert points on the map passing a list of latitudes and longitudes
gmap.heatmap(coordinates['latitude'], coordinates['longitude'] ,max_intensity=300,radius=20)

# Save the map to html file
gmap.draw("python_heatmap.html")

In [ ]:
wordcloud?

In [ ]:
## hashtags

In [ ]:
# NEXT STEPS LOOKING AT CSV FILE
import pandas as pd
tweets = pd.read_csv("saved_tweets.csv",header=None)
tweets=tweets[:]
tweets=tweets.drop_duplicates() ## drop duplicate rows
len(tweets)

In [ ]:
from collections import Counter
import ast

tweets = pd.read_csv("saved_tweets.csv",header=None)

# Extract hashtags and put them in a list
list_hashtag_strings = [entry for entry in tweets[0]]
list_hashtag_lists = ast.literal_eval(','.join(list_hashtag_strings))
hashtag_list = [ht.lower() for list_ in list_hashtag_lists for ht in list_]

# Count most common hashtags
counter_hashtags = Counter(hashtag_list)
#counter_hashtags.most_common(50)
common=counter_hashtags.most_common(100)

In [ ]:
import pandas as pd
import numpy as np

#for plotting images & adjusting colors
import matplotlib.pyplot as plt
import matplotlib as mpl
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
#convert into dict
common= dict(common)

In [ ]:
#font="gargi.ttf"
from wordcloud import WordCloud
wc = WordCloud(background_color='black', width = 400, height=300, margin=8)

wc.fit_words(common)
wc.to_file('wc1.png')

plt.figure(figsize=[25,25])
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

clean_tweets_nsw = pd.DataFrame(counter_hashtags.most_common(30),
                             columns=['words', 'count'])

fig, ax = plt.subplots(figsize=(8, 8))

# Plot horizontal bar graph
clean_tweets_nsw.sort_values(by='count').plot.barh(x='words',
                      y='count',
                      ax=ax,
                      color="purple")

ax.set_title("Hastags Trending on Twitter #farmersprotest")

plt.show()

In [ ]:
counter_hashtags.most_common(30)